In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import sqlglot

In [22]:
sql = 'SELECT a, avg(b) FROM mytable WHERE a > 10 AND a < 15 OR a > 1000 GROUP BY a'

In [23]:
parsed = sqlglot.parse(sql, 'postgres')[0]  # sqlglot.parse(sql) returns list, every element is sql statement if I understand correctly
parsed

(SELECT expressions: 
  (COLUMN this: 
    (IDENTIFIER this: a, quoted: False)), 
  (AVG this: 
    (COLUMN this: 
      (IDENTIFIER this: b, quoted: False))), from: 
  (FROM expressions: 
    (TABLE this: 
      (IDENTIFIER this: mytable, quoted: False))), where: 
  (WHERE this: 
    (OR this: 
      (AND this: 
        (GT this: 
          (COLUMN this: 
            (IDENTIFIER this: a, quoted: False)), expression: 
          (LITERAL this: 10, is_string: False)), expression: 
        (LT this: 
          (COLUMN this: 
            (IDENTIFIER this: a, quoted: False)), expression: 
          (LITERAL this: 15, is_string: False))), expression: 
      (GT this: 
        (COLUMN this: 
          (IDENTIFIER this: a, quoted: False)), expression: 
        (LITERAL this: 1000, is_string: False)))), group: 
  (GROUP expressions: 
    (COLUMN this: 
      (IDENTIFIER this: a, quoted: False))))

In [24]:
type(parsed)

sqlglot.expressions.Select

In [25]:
for x in parsed.expressions:
    print(type(x))

<class 'sqlglot.expressions.Column'>
<class 'sqlglot.expressions.Avg'>


In [26]:
from sqlglot.expressions import Where

where = parsed.find(Where)
where

(WHERE this: 
  (OR this: 
    (AND this: 
      (GT this: 
        (COLUMN this: 
          (IDENTIFIER this: a, quoted: False)), expression: 
        (LITERAL this: 10, is_string: False)), expression: 
      (LT this: 
        (COLUMN this: 
          (IDENTIFIER this: a, quoted: False)), expression: 
        (LITERAL this: 15, is_string: False))), expression: 
    (GT this: 
      (COLUMN this: 
        (IDENTIFIER this: a, quoted: False)), expression: 
      (LITERAL this: 1000, is_string: False))))

In [27]:
where

(WHERE this: 
  (OR this: 
    (AND this: 
      (GT this: 
        (COLUMN this: 
          (IDENTIFIER this: a, quoted: False)), expression: 
        (LITERAL this: 10, is_string: False)), expression: 
      (LT this: 
        (COLUMN this: 
          (IDENTIFIER this: a, quoted: False)), expression: 
        (LITERAL this: 15, is_string: False))), expression: 
    (GT this: 
      (COLUMN this: 
        (IDENTIFIER this: a, quoted: False)), expression: 
      (LITERAL this: 1000, is_string: False))))

In [28]:
and_ = where.this
and_

(OR this: 
  (AND this: 
    (GT this: 
      (COLUMN this: 
        (IDENTIFIER this: a, quoted: False)), expression: 
      (LITERAL this: 10, is_string: False)), expression: 
    (LT this: 
      (COLUMN this: 
        (IDENTIFIER this: a, quoted: False)), expression: 
      (LITERAL this: 15, is_string: False))), expression: 
  (GT this: 
    (COLUMN this: 
      (IDENTIFIER this: a, quoted: False)), expression: 
    (LITERAL this: 1000, is_string: False)))

In [29]:
and_

(OR this: 
  (AND this: 
    (GT this: 
      (COLUMN this: 
        (IDENTIFIER this: a, quoted: False)), expression: 
      (LITERAL this: 10, is_string: False)), expression: 
    (LT this: 
      (COLUMN this: 
        (IDENTIFIER this: a, quoted: False)), expression: 
      (LITERAL this: 15, is_string: False))), expression: 
  (GT this: 
    (COLUMN this: 
      (IDENTIFIER this: a, quoted: False)), expression: 
    (LITERAL this: 1000, is_string: False)))

In [30]:
for k, v in parsed.args.items():
    print(k, v)

hint None
distinct None
expressions [(COLUMN this: 
  (IDENTIFIER this: a, quoted: False)), (AVG this: 
  (COLUMN this: 
    (IDENTIFIER this: b, quoted: False)))]
limit None
from FROM mytable
where WHERE a > 10 AND a < 15 OR a > 1000
group GROUP BY a


In [31]:
from sqlglot import Expression
from sqlglot.helper import ensure_collection

for k, v in parsed.args.items():
    for node in ensure_collection(v):
        if isinstance(node, Expression):
            print(k, node)

expressions a
expressions AVG(b)
from FROM mytable
where WHERE a > 10 AND a < 15 OR a > 1000
group GROUP BY a


In [32]:
from sqlglot import Expression
from sqlglot.helper import ensure_collection

start_node = where.this
print(type(start_node))
for k, v in start_node.args.items():
    for node in ensure_collection(v):
        if isinstance(node, Expression):
            print(k, node)

<class 'sqlglot.expressions.Or'>
this a > 10 AND a < 15
expression a > 1000


In [33]:
#node: Expression
for node in where.this.dfs():
    print(node[0])


a > 10 AND a < 15 OR a > 1000
a > 10 AND a < 15
a > 10
a
a
10
a < 15
a
a
15
a > 1000
a
a
1000


In [34]:
where

(WHERE this: 
  (OR this: 
    (AND this: 
      (GT this: 
        (COLUMN this: 
          (IDENTIFIER this: a, quoted: False)), expression: 
        (LITERAL this: 10, is_string: False)), expression: 
      (LT this: 
        (COLUMN this: 
          (IDENTIFIER this: a, quoted: False)), expression: 
        (LITERAL this: 15, is_string: False))), expression: 
    (GT this: 
      (COLUMN this: 
        (IDENTIFIER this: a, quoted: False)), expression: 
      (LITERAL this: 1000, is_string: False))))

In [40]:
import where_parser

where_func = where_parser.parse_where_simple(where.this)
where_func


def _parse_where_simple_inner(a):
    return a > 10 and a < 15 or a > 1000



<function where_parser._parse_where_simple_inner(a)>

In [44]:
where_func(1210)

True